### KMS Validation with image classification

In [ ]:
import os
import tensorflow as tf
import numpy as np

In [ ]:
!git clone https://github.com/MamiSiaga/MachineLearning.git

In [ ]:
# Directory with our training dandelion pictures
train_kms_dir = '/content/MachineLearning/Data/KMS_validation/train/kms'

# Directory with our training grass pictures
train_not_dir = '/content/MachineLearning/Data/KMS_validation/train/not kms'

# Directory with our validation dandelion pictures
valid_kms_dir = '/content/MachineLearning/Data/KMS_validation/validation/kms'

# Directory with our validation grass pictures
valid_not_dir = '/content/MachineLearning/Data/KMS_validation/validation/not kms'

In [ ]:
train_kms_names = os.listdir(train_kms_dir)
print(train_kms_names[:10])

train_not_names = os.listdir(train_not_dir)
print(train_not_names[:10])

validation_kms_names = os.listdir(valid_kms_dir)
print(validation_kms_names[:10])

validation_not_names = os.listdir(valid_not_dir)
print(validation_not_names[:10])

In [ ]:
print('total training kms images:', len(os.listdir(train_kms_dir)))
print('total training not kms images:', len(os.listdir(train_not_dir)))
print('total validation kms images:', len(os.listdir(valid_kms_dir)))
print('total validation not kms images:', len(os.listdir(valid_not_dir)))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_kms_fnames = os.listdir(train_kms_dir)
train_not_fnames = os.listdir(train_not_dir)

train_dir = '/content/MachineLearning/Data/KMS_validation/train'
validation_dir = '/content/MachineLearning/Data/KMS_validation/validation'

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1/255.)
# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale = 1/255.)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 1,
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                          batch_size  = 1,
                                                          target_size = (150, 150))

In [ ]:
validation_generator.class_indices

In [ ]:
from tensorflow import keras

tf.keras.backend.clear_session()
model = keras.Sequential()

# Convolutional layer and maxpool layer 1
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 2
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 3
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 4
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# This layer flattens the resulting image array to 1D array
model.add(keras.layers.Flatten())

# Hidden layer with 512 neurons and Rectified Linear Unit activation function 
model.add(keras.layers.Dense(512,activation='relu'))

# Output layer with single neuron which gives 0 for Cat or 1 for Dog 
#Here we use sigmoid activation function which makes our model output to lie between 0 and 1
model.add(keras.layers.Dense(2, activation='softmax'))

model.summary()

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_generator,
                              epochs = 10,
                              validation_data = validation_generator)

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
def predict(path):
    image_path = path

    img = tf.keras.preprocessing.image.load_img(
        image_path, target_size=(150, 150)
    )
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    images = np.vstack([img_array])

    predictions = model.predict(images)

    print(predictions)

In [ ]:
predict("/content/MachineLearning/images/image1.jpg")

In [ ]:
predict("/content/MachineLearning/images/image4.jpg")

In [ ]:
predict('/content/MachineLearning/Data/KMS_validation/validation/not kms/not1.jpg')

In [ ]:
predict('/content/MachineLearning/Data/KMS_validation/validation/not kms/not2.jpg')

## Save Model

In [ ]:
model.save('/content/MachineLearning/Model')

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('/content/MachineLearning/Model') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('/content/MachineLearning/Model/model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
import matplotlib.pyplot as plt


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Train')
plt.plot(epochs, val_acc, 'b', label='Val')
plt.title('model accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
plt.plot(epochs, loss, 'r', label='Train')
plt.plot(epochs, val_loss, 'b', label='Val')
plt.title('model loss')
plt.legend(loc=0)
plt.figure()


plt.show()